In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt


In [4]:
# Define the autoencoder model
def build_autoencoder(input_shape):
    model = keras.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')
    ])
    return model
IMAGE_SIZE = 256
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)  # Example input shape
autoencoder = build_autoencoder(input_shape)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')


C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
        'dataset/train',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse",
)

Found 1506 images belonging to 3 classes.


In [6]:
def autoencoder_generator(generator):
    for batch in generator:
        yield batch, batch  # Input = Output

# Use the wrapped generator
train_gen = autoencoder_generator(train_generator)

In [11]:


# Fit the autoencoder
autoencoder.fit(train_gen,
                steps_per_epoch=47,  # Adjust as needed
                epochs=30,
                verbose=1)


Epoch 1/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - loss: 0.0309
Epoch 2/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - loss: 0.0069
Epoch 3/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - loss: 0.0046
Epoch 4/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 45s 954ms/step - loss: 0.0038
Epoch 5/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 41s 879ms/step - loss: 0.0037
Epoch 6/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - loss: 0.0033
Epoch 7/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 35s 750ms/step - loss: 0.0032
Epoch 8/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 35s 739ms/step - loss: 0.0030
Epoch 9/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 35s 753ms/step - loss: 0.0030
Epoch 10/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 34s 734ms/step - loss: 0.0028
Epoch 11/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 35s 749ms/step - loss: 0.0026
Epoch 12/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 35s 741ms/step - loss: 0.0026
Epoch 13/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 35s 740ms/step - loss: 0.0025
Epoch 14/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 593s 13s/step - loss: 0.0025
Epoch 15/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 37s 784ms/step - 

In [7]:
autoencoder.save("../saved-models/modelauto.keras")
autoencoder.save('../saved-models/autoencoder_model.h5')

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

autoencoder = load_model('../saved-models/autoencoder_model.h5')


In [8]:
#  Define a function to compute reconstruction error
def compute_reconstruction_error(model, images):
    reconstructed = model.predict(images, verbose=0)  
    return np.mean(np.abs(reconstructed - images), axis=(1, 2, 3))

IMG_SIZE = 256
BATCH_SIZE = 32

test_datagen = ImageDataGenerator(rescale=1./255) 

train_generator = test_datagen.flow_from_directory(
    'dataset/train',  
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode=None 
)

reconstruction_errors = []

for batch in train_generator:
    batch_errors = compute_reconstruction_error(autoencoder, batch)
    reconstruction_errors.extend(batch_errors)
    if len(reconstruction_errors) >= train_generator.samples:
        break 


reconstruction_errors = np.array(reconstruction_errors)
np.save('reconstruction_errors.npy', reconstruction_errors)  

# Set the threshold (95th percentile of errors)
threshold = np.percentile(reconstruction_errors, 95)
print(f"Reconstruction error threshold: {threshold}")

Found 1506 images belonging to 3 classes.
Reconstruction error threshold: 0.19881732761859894


In [5]:
def predict_with_autoencoder(model, img, threshold):
    
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0 
    img_array = np.expand_dims(img_array, axis=0) 

    # Compute reconstruction error for the input image
    reconstruction_error = compute_reconstruction_error(model, img_array)[0]

    # Return the prediction result
    if reconstruction_error > threshold:
        return "I don't know this image"
    else:
        return "This image is recognized."

In [6]:
from keras.preprocessing.image import load_img

test_image_path = './paris.jpeg '

test_image = load_img(test_image_path, target_size=(IMG_SIZE, IMG_SIZE))

prediction = predict_with_autoencoder(autoencoder, test_image, threshold)
print(f"Prediction: {prediction}")

Prediction: I don't know this image
